<p style="font-size:30px; text-align:center; line-height:120%">
    <br> 
        <b>
        COMS 4995 Applied ML
            Homework 4 
        <br></br>
            Predicting Wine Quality: Task 3
        <br></br>
        </b> 
    <br> 
</p>
<p style="font-size:18px; text-align:left; line-height:120%">
    <br> 
        <b>
        Kirit Dhillon, Sagar Lal
        </b>
    <br> 
        <b>
        Uni: ksd2142, sl3946
        </b>
</p>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Data Loading and Exploration

In [2]:
data_main = pd.read_csv("winemag-data-130k-v2.csv")
data = data_main.sample(frac=0.1)
# Remove uninformative columns like "Taster Name" and "Taster Twitter Handle"
data = data.drop(['taster_name', 'taster_twitter_handle'], axis=1)

In [3]:
from sklearn.model_selection import train_test_split
X_trainval, X_test, y_trainval, y_test = train_test_split(data['description'], data['points'], stratify= data['points'])

In [4]:
print("X_trainval: \t", X_trainval.shape, y_trainval.shape)
print("X_test: \t", X_test.shape, y_test.shape)

X_trainval: 	 (9747,) (9747,)
X_test: 	 (3250,) (3250,)


### BERT Based Model

In [5]:
from transformers import pipeline
nlp = pipeline("feature-extraction")

In [6]:
list_X_trainval = X_trainval.values.tolist()
list_X_test = X_test.values.tolist()
print(X_trainval.shape)
print(X_test.shape)

(9747,)
(3250,)


In [8]:
def nlp_preprocess(dataset):
    nlp_arr = []
    j = 0
    for i in dataset:
        j+=1
        if j%1000 == 0:
            print(j)
        nlp_arr.append(nlp(i))
    return nlp_arr

In [9]:
nlp_X_trainval_preprocess = nlp_preprocess(list_X_trainval)
print("Completed X_trainval preprocessing")

1000
2000
3000
4000
5000
6000
7000
8000
9000
Completed X_trainval preprocessing


In [10]:
%%time
nlp_X_test_preprocess = nlp_preprocess(list_X_test)
print("Completed X_test preprocessing")


1000
2000
3000
Completed X_test preprocessing
CPU times: user 9min 48s, sys: 2min, total: 11min 49s
Wall time: 8min 36s


In [11]:
# Convert lists to Pandas dataframe to allow for concatenation with non-text features later
nlp_X_trainval = pd.DataFrame(nlp_X_trainval_preprocess)
nlp_X_test = pd.DataFrame(nlp_X_test_preprocess)

In [12]:
# Debugging: 
print("X_trainval: \t", nlp_X_trainval.shape)
print("X_test: \t", nlp_X_test.shape)

X_trainval: 	 (9747, 1)
X_test: 	 (3250, 1)


In [13]:
# Write dataframes to CSVs for future-use
nlp_X_trainval.to_csv("./nlp_X_trainval.csv")
nlp_X_test.to_csv("./nlp_X_test.csv")

In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor

In [15]:
%%time
ridge_pipeline = Pipeline(steps=[
                                ('regressor', Ridge())])

param_grid =  {
               'regressor__alpha': [1]
              }
grid = GridSearchCV(ridge_pipeline, param_grid, cv=3, return_train_score=True)
grid.fit(nlp_X_trainval, y_trainval)

print(("BERT Ridge score with GridSearchCV %.2f"
       %grid.score(nlp_X_test, y_test)))
print("Best param:", grid.best_params_)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:542: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


ValueError: setting an array element with a sequence.